# CUDA (Grid-Stride Loop)

In [3]:
import os

# Add the directory containing the executable to the PATH
os.environ["PATH"] += os.pathsep + "/usr/local/cuda/bin"

# Check if the directory is added to the PATH
print(os.environ["PATH"])

/opt/tljh/user/bin:/bin:/usr/bin:/usr/local/cuda/bin


In [17]:
%%writefile CUDA_max.cu
#include <stdio.h>
#include <stdlib.h>

#define MIN_VAL -20
#define MAX_VAL 20

__global__
void cuda_max(size_t n, float* max_arr, float *A, float *B, int* idx){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride){
        max_arr[i] = (A[i] > B[i]) ? A[i] : B[i];
        idx[i] = (A[i] >= B[i]) ? 0 : 1;
    }
}

int main(){
  const size_t ARRAY_SIZE = 1<<24;
  const size_t ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
 //number of times the program is to be executed
   const size_t loope = 1;
//declare array
  float *A, *B, *max_arr;
  int *idx;

  cudaMallocManaged(&A, ARRAY_BYTES);
  cudaMallocManaged(&B, ARRAY_BYTES);
  cudaMallocManaged(&max_arr, ARRAY_BYTES);
  cudaMallocManaged(&idx, ARRAY_BYTES);
// *** init array
  for (size_t i=0; i<ARRAY_SIZE; i++){
     A[i] = MIN_VAL + ((float) rand() / (float) RAND_MAX) * (MAX_VAL - MIN_VAL);
     B[i] = MIN_VAL + ((float) rand() / (float) RAND_MAX) * (MAX_VAL - MIN_VAL);
  }
// *** setup CUDA kernel
    size_t numThreads = 1024;
    size_t numBlocks = (ARRAY_SIZE + numThreads-1) / numThreads;
  printf("*** function = DAXPY\n");
  printf("numElements = %lu\n", ARRAY_SIZE);
  printf("numBlocks = %lu, numThreads = %lu \n",numBlocks, numThreads);
  for (size_t i=0; i<loope;i++)
    cuda_max <<<numBlocks, numThreads>>> (ARRAY_SIZE,max_arr,A,B, idx);
//barrier
    cudaDeviceSynchronize();
//error checking
  size_t err_count = 0;
  for (size_t i=0; i<ARRAY_SIZE; i++){
    if(((A[i] > B[i]) ? A[i] : B[i]) != max_arr[i])
      err_count++;
  }
  printf("Error count(CUDA program): %zu\n", err_count);
//free memory
  cudaFree(A);
  cudaFree(B);
  cudaFree(max_arr);
  cudaFree(idx);
  return 0;
}

Overwriting CUDA_max.cu


In [18]:
%%bash
nvcc CUDA_max.cu -o CUDA_max -Wno-deprecated-gpu-targets

In [20]:
%%bash
nvprof ./CUDA_max

==892676== NVPROF is profiling process 892676, command: ./CUDA_max


*** function = DAXPY
numElements = 16777216
numBlocks = 16384, numThreads = 1024 
Error count(CUDA program): 0


==892676== Profiling application: ./CUDA_max
==892676== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  101.38ms         1  101.38ms  101.38ms  101.38ms  cuda_max(unsigned long, float*, float*, float*, int*)
      API calls:   85.69%  1.01271s         4  253.18ms  44.573us  1.01212s  cudaMallocManaged
                    8.58%  101.40ms         1  101.40ms  101.40ms  101.40ms  cudaDeviceSynchronize
                    3.70%  43.734ms         1  43.734ms  43.734ms  43.734ms  cudaLaunchKernel
                    1.95%  23.052ms         3  7.6842ms  6.3322ms  10.112ms  cudaFree
                    0.06%  678.59us       114  5.9520us     190ns  270.28us  cuDeviceGetAttribute
                    0.02%  191.96us         1  191.96us  191.96us  191.96us  cuDeviceGetName
                    0.00%  53.679us         1  53.679us  53.679us  53.679us  cuDeviceTotalMem
                    0.00%  21.673us         1  21.673u

# CUDA (Grid-Stride Loop + Prefetching)

In [25]:
%%writefile CUDA_max2.cu
#include <stdio.h>
#include <stdlib.h>

#define MIN_VAL -20
#define MAX_VAL 20

__global__
void cuda_max(size_t n, float* max_arr, float *A, float *B, int* idx){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride){
        max_arr[i] = (A[i] > B[i]) ? A[i] : B[i];
        idx[i] = (A[i] >= B[i]) ? 0 : 1;
    }
}

int main(){
  const size_t ARRAY_SIZE = 1<<24;
  const size_t ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
 //number of times the program is to be executed
   const size_t loope = 1;
//declare array
  float *A, *B, *max_arr;
  int *idx;

  cudaMallocManaged(&A, ARRAY_BYTES);
  cudaMallocManaged(&B, ARRAY_BYTES);
  cudaMallocManaged(&max_arr, ARRAY_BYTES);
  cudaMallocManaged(&idx, ARRAY_BYTES);
//get gpu id
  int device = -1;
  cudaGetDevice(&device);
// *** init array
  for (size_t i=0; i<ARRAY_SIZE; i++){
     A[i] = MIN_VAL + ((float) rand() / (float) RAND_MAX) * (MAX_VAL - MIN_VAL);
     B[i] = MIN_VAL + ((float) rand() / (float) RAND_MAX) * (MAX_VAL - MIN_VAL);
  }
cudaMemPrefetchAsync(A,ARRAY_BYTES,device,NULL);
cudaMemPrefetchAsync(B,ARRAY_BYTES,device,NULL);
// *** setup CUDA kernel
    size_t numThreads = 1024;
    size_t numBlocks = (ARRAY_SIZE + numThreads-1) / numThreads;
  printf("*** function = MAX\n");
  printf("numElements = %lu\n", ARRAY_SIZE);
  printf("numBlocks = %lu, numThreads = %lu \n",numBlocks, numThreads);
  for (size_t i=0; i<loope;i++)
    cuda_max <<<numBlocks, numThreads>>> (ARRAY_SIZE,max_arr,A,B, idx);
//barrier
    cudaDeviceSynchronize();

  cudaMemPrefetchAsync(max_arr,ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(A,ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(B,ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(idx,ARRAY_BYTES,cudaCpuDeviceId,NULL);

//error checking
  size_t err_count = 0;
  for (size_t i=0; i<ARRAY_SIZE; i++){
    if(((A[i] > B[i]) ? A[i] : B[i]) != max_arr[i])
      err_count++;
  }
  printf("Error count(CUDA program): %zu\n", err_count);
//free memory
  cudaFree(A);
  cudaFree(B);
  cudaFree(max_arr);
  cudaFree(idx);
  return 0;
}

Overwriting CUDA_max2.cu


In [26]:
%%bash
nvcc CUDA_max2.cu -o CUDA_max2 -Wno-deprecated-gpu-targets

In [27]:
%%bash
nvprof ./CUDA_max2

==894344== NVPROF is profiling process 894344, command: ./CUDA_max2


*** function = DAXPY
numElements = 16777216
numBlocks = 16384, numThreads = 1024 
Error count(CUDA program): 0


==894344== Profiling application: ./CUDA_max2
==894344== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  35.988ms         1  35.988ms  35.988ms  35.988ms  cuda_max(unsigned long, float*, float*, float*, int*)
      API calls:   82.48%  1.32440s         4  331.10ms  73.951us  1.32353s  cudaMallocManaged
                   14.14%  227.07ms         6  37.845ms  9.8566ms  80.074ms  cudaMemPrefetchAsync
                    2.25%  36.058ms         1  36.058ms  36.058ms  36.058ms  cudaDeviceSynchronize
                    0.93%  14.939ms         4  3.7348ms  3.1393ms  4.1942ms  cudaFree
                    0.15%  2.3542ms         1  2.3542ms  2.3542ms  2.3542ms  cudaLaunchKernel
                    0.03%  549.31us       114  4.8180us     143ns  203.00us  cuDeviceGetAttribute
                    0.02%  296.27us         1  296.27us  296.27us  296.27us  cuDeviceGetName
                    0.00%  28.586us         1  28

# CUDA (Grid-Stride Loop + Prefetching + Page Creation)

In [50]:
%%writefile CUDA_max3.cu
#include <stdio.h>
#include <stdlib.h>

#define MIN_VAL -20
#define MAX_VAL 20

__global__
void cuda_max(size_t n, float* max_arr, float *A, float *B, int* idx){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride){
        max_arr[i] = (A[i] > B[i]) ? A[i] : B[i];
        idx[i] = (A[i] >= B[i]) ? 0 : 1;
    }
}

int main(){
  const size_t ARRAY_SIZE = 1<<24;
  const size_t ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
 //number of times the program is to be executed
   const size_t loope = 1;
//declare array
  float *A, *B, *max_arr;
  int *idx;
  cudaMallocManaged(&A, ARRAY_BYTES);
  cudaMallocManaged(&B, ARRAY_BYTES);
  cudaMallocManaged(&max_arr, ARRAY_BYTES);
  cudaMallocManaged(&idx, ARRAY_BYTES);

//get gpu id
  int device = -1;
  cudaGetDevice(&device);
//"prefetch data" to create CPU page memory
  cudaMemPrefetchAsync(A,ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(B,ARRAY_BYTES,cudaCpuDeviceId,NULL);
//"prefetch data" to create GPU page memory
  cudaMemPrefetchAsync(max_arr,ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(idx,ARRAY_BYTES,device,NULL);

// *** init array
  for (size_t i=0; i<ARRAY_SIZE; i++){
     A[i] = MIN_VAL + ((float) rand() / (float) RAND_MAX) * (MAX_VAL - MIN_VAL);
     B[i] = MIN_VAL + ((float) rand() / (float) RAND_MAX) * (MAX_VAL - MIN_VAL);
  }

cudaMemPrefetchAsync(A,ARRAY_BYTES,device,NULL);
cudaMemPrefetchAsync(B,ARRAY_BYTES,device,NULL);
// *** setup CUDA kernel
    size_t numThreads = 1024;
    size_t numBlocks = (ARRAY_SIZE + numThreads-1) / numThreads;
  printf("*** function = MAX\n");
  printf("numElements = %lu\n", ARRAY_SIZE);
  printf("numBlocks = %lu, numThreads = %lu \n",numBlocks, numThreads);
  for (size_t i=0; i<loope;i++)
    cuda_max <<<numBlocks, numThreads>>> (ARRAY_SIZE,max_arr,A,B, idx);
//barrier
    cudaDeviceSynchronize();

  cudaMemPrefetchAsync(max_arr,ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(A,ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(B,ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(idx,ARRAY_BYTES,cudaCpuDeviceId,NULL);

//error checking
  size_t err_count = 0;
  for (size_t i=0; i<ARRAY_SIZE; i++){
    if(((A[i] > B[i]) ? A[i] : B[i]) != max_arr[i])
      err_count++;
  }
  printf("Error count(CUDA program): %zu\n", err_count);
//free memory
  cudaFree(A);
  cudaFree(B);
  cudaFree(max_arr);
  cudaFree(idx);
  return 0;
}

Overwriting CUDA_max3.cu


In [51]:
%%bash
nvcc CUDA_max3.cu -o CUDA_max3 -Wno-deprecated-gpu-targets

In [52]:
%%bash
nvprof ./CUDA_max3

==896838== NVPROF is profiling process 896838, command: ./CUDA_max3


*** function = MAX
numElements = 1024
numBlocks = 1, numThreads = 1024 
Error count(CUDA program): 0


==896838== Profiling application: ./CUDA_max3
==896838== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  3.4880us         1  3.4880us  3.4880us  3.4880us  cuda_max(unsigned long, float*, float*, float*, int*)
      API calls:   99.37%  882.08ms         4  220.52ms  6.0900us  881.89ms  cudaMallocManaged
                    0.34%  3.0067ms        10  300.67us  39.198us  1.9071ms  cudaMemPrefetchAsync
                    0.12%  1.0534ms         1  1.0534ms  1.0534ms  1.0534ms  cudaLaunchKernel
                    0.07%  647.45us       114  5.6790us     141ns  273.18us  cuDeviceGetAttribute
                    0.07%  621.94us         4  155.48us  27.865us  472.47us  cudaFree
                    0.02%  178.47us         1  178.47us  178.47us  178.47us  cuDeviceGetName
                    0.00%  28.086us         1  28.086us  28.086us  28.086us  cuDeviceTotalMem
                    0.00%  17.132us         1  17.132u

# CUDA (Grid-Stride Loop + Prefetching + Page Creation + Mem Advise)

In [53]:
%%writefile CUDA_max4.cu
#include <stdio.h>
#include <stdlib.h>

#define MIN_VAL -20
#define MAX_VAL 20

__global__
void cuda_max(size_t n, float* max_arr, float *A, float *B, int* idx){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride){
        max_arr[i] = (A[i] > B[i]) ? A[i] : B[i];
        idx[i] = (A[i] >= B[i]) ? 0 : 1;
    }
}

int main(){
  const size_t ARRAY_SIZE = 1<<24;
  const size_t ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
 //number of times the program is to be executed
   const size_t loope = 1;
//declare array
  float *A, *B, *max_arr;
  int *idx;
  cudaMallocManaged(&A, ARRAY_BYTES);
  cudaMallocManaged(&B, ARRAY_BYTES);
  cudaMallocManaged(&max_arr, ARRAY_BYTES);
  cudaMallocManaged(&idx, ARRAY_BYTES);

//get gpu id
  int device = -1;
  cudaGetDevice(&device);
//mem advise
  cudaMemAdvise(A, ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
  cudaMemAdvise(A, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);
  cudaMemAdvise(B, ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
  cudaMemAdvise(B, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);

//"prefetch data" to create CPU page memory
  cudaMemPrefetchAsync(A,ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(B,ARRAY_BYTES,cudaCpuDeviceId,NULL);
//"prefetch data" to create GPU page memory
  cudaMemPrefetchAsync(max_arr,ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(idx,ARRAY_BYTES,device,NULL);

// *** init array
  for (size_t i=0; i<ARRAY_SIZE; i++){
     A[i] = MIN_VAL + ((float) rand() / (float) RAND_MAX) * (MAX_VAL - MIN_VAL);
     B[i] = MIN_VAL + ((float) rand() / (float) RAND_MAX) * (MAX_VAL - MIN_VAL);
  }

cudaMemPrefetchAsync(A,ARRAY_BYTES,device,NULL);
cudaMemPrefetchAsync(B,ARRAY_BYTES,device,NULL);

// *** setup CUDA kernel
    size_t numThreads = 1024;
    size_t numBlocks = (ARRAY_SIZE + numThreads-1) / numThreads;
  printf("*** function = MAX\n");
  printf("numElements = %lu\n", ARRAY_SIZE);
  printf("numBlocks = %lu, numThreads = %lu \n",numBlocks, numThreads);
  for (size_t i=0; i<loope;i++)
    cuda_max <<<numBlocks, numThreads>>> (ARRAY_SIZE,max_arr,A,B, idx);
//barrier
    cudaDeviceSynchronize();

  cudaMemPrefetchAsync(max_arr,ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(A,ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(B,ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(idx,ARRAY_BYTES,cudaCpuDeviceId,NULL);

//error checking
  size_t err_count = 0;
  for (size_t i=0; i<ARRAY_SIZE; i++){
    if(((A[i] > B[i]) ? A[i] : B[i]) != max_arr[i])
      err_count++;
  }
  printf("Error count(CUDA program): %zu\n", err_count);
//free memory
  cudaFree(A);
  cudaFree(B);
  cudaFree(max_arr);
  cudaFree(idx);
  return 0;
}

Overwriting CUDA_max4.cu


In [54]:
%%bash
nvcc CUDA_max4.cu -o CUDA_max4 -Wno-deprecated-gpu-targets

In [55]:
%%bash
nvprof ./CUDA_max4

==896892== NVPROF is profiling process 896892, command: ./CUDA_max4


*** function = MAX
numElements = 1024
numBlocks = 1, numThreads = 1024 
Error count(CUDA program): 0


==896892== Profiling application: ./CUDA_max4
==896892== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  3.8400us         1  3.8400us  3.8400us  3.8400us  cuda_max(unsigned long, float*, float*, float*, int*)
      API calls:   99.63%  974.58ms         4  243.64ms  7.0280us  974.39ms  cudaMallocManaged
                    0.13%  1.2510ms        10  125.10us  24.693us  332.57us  cudaMemPrefetchAsync
                    0.09%  926.75us         1  926.75us  926.75us  926.75us  cudaLaunchKernel
                    0.07%  676.74us         4  169.19us  28.727us  509.81us  cudaFree
                    0.04%  427.46us       114  3.7490us     107ns  213.69us  cuDeviceGetAttribute
                    0.02%  162.67us         4  40.666us  9.0430us  132.38us  cudaMemAdvise
                    0.01%  127.09us         1  127.09us  127.09us  127.09us  cuDeviceGetName
                    0.00%  26.263us         1  26.263us  

In [17]:
%%bash
nsys profile  -o CUDA_max4 ./CUDA_max4

         This may increase runtime overhead and the likelihood of false
         dependencies across CUDA Streams. If you wish to avoid this, please
         disable the feature with --cuda-event-trace=false.
Try the 'nsys status --environment' command to learn more.

Try the 'nsys status --environment' command to learn more.



*** function = DAXPY
numElements = 16777216
numBlocks = 16384, numThreads = 1024 
Error count(CUDA program): 0


Failed to create '/home/jupyter-benn_llovit@dlsu.e-0b35d/DAXPY_CUDA4.nsys-rep': File exists.
Use `--force-overwrite true` to overwrite existing files.


Generating '/tmp/nsys-report-2eb7.qdstrm'
[1/1] [========================100%] nsys-report-2791.nsys-rep
Generated:
	/tmp/nsys-report-2791.nsys-rep
